# Imports

In [2]:
from pyiron_contrib.tinybase.task import AbstractTask, FunctionTask, SeriesTask, LoopTask

In [3]:
from pyiron_contrib.tinybase.executor import ProcessExecutor, BackgroundExecutor, Executor

In [4]:
import logging
logging.getLogger().setLevel(20)

In [5]:
import numpy as np

# Function Task

## Basic

In [6]:
def calc_fib(n):
    import time
    n1 = n2 = 1
    for i in range(n):
        time.sleep(.1)
        x = n1 + n2
        n1 = n2
        n2 = x
    return x

In [7]:
f = FunctionTask(calc_fib)

In [8]:
f.input.storage

DataContainer([])

In [9]:
f.input.args

[]

In [10]:
f.input.kwargs

{}

In [11]:
f.input.kwargs['n'] = 10

In [12]:
f.input.kwargs

{'n': 10}

In [13]:
exe = f.run()

In [14]:
exe._run_machine._data

{'status': (ReturnStatus(Code.DONE, None),),
 'output': (<pyiron_contrib.tinybase.task.FunctionOutput at 0x7f4f3f730b20>,)}

In [15]:
exe.output[0].result

144

In [16]:
exe._run_time, exe._collect_time

(1.0841432279994478, 3.1526011298410594e-05)

## We don't actually have to use an executor if we just want a result

In [17]:
f = FunctionTask(calc_fib)

In [18]:
f.input.kwargs['n'] = 10

In [19]:
f.execute()

(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7f4f3f731510>)

## Do the same but in the background

In [20]:
f = FunctionTask(calc_fib)

In [21]:
f.input.kwargs['n'] = 100

In [22]:
exe = f.run(how='background')

In [23]:
exe._run_machine._data

{}

In [24]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [25]:
exe.wait()

In [26]:
exe.output[0].result

927372692193078999176

## Do the same but in the background as process

In [27]:
fib_Task = FunctionTask(calc_fib)

In [28]:
fib_Task.input.kwargs['n'] = 100

In [29]:
exe = fib_Task.run(how='process')

In [30]:
exe._run_machine._data

{}

In [31]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [32]:
exe.wait()

In [33]:
exe.output[0].result

927372692193078999176

# Executors handle single Tasks and lists of them on the same footing

In [35]:
tasks = [FunctionTask(calc_fib) for _ in range(10)]

In [49]:
for i, n in enumerate(tasks):
    n.input.kwargs['n'] = 3 + i

## With the basic executor

In [51]:
exe = Executor(tasks)
exe.run()

In [52]:
exe.output

(<pyiron_contrib.tinybase.task.FunctionOutput at 0x7f4f3f7155d0>,
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7f4f3f716950>)

In [53]:
exe.output[1].result

8

## With the process executor

In [54]:
exe = ProcessExecutor(tasks)
exe.run()

In [55]:
exe.wait()

In [56]:
exe.status

[ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None)]

In [57]:
exe.output

In [58]:
exe.output[5].result

55

# SeriesTask

In [59]:
s = SeriesTask()

In [60]:
f1 = FunctionTask(calc_fib)

In [61]:
f2 = FunctionTask(np.sqrt)

In [62]:
def transfer(input, output):
    input.args = [output.result]

In [63]:
s.input.first(f1).then(f2, transfer)

In [65]:
s.input.tasks[0].input.kwargs['n'] = 10

In [66]:
status, output = s.execute()

In [67]:
status

ReturnStatus(Code.DONE, None)

In [68]:
output.result

12.0

# Loop Task

## Simple repeat loop

In [69]:
l = LoopTask()

In [71]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [72]:
l.input.repeat(10, restart=lambda output, input, scratch: print(output.result))

In [73]:
exe = l.run()

0.9953802852390651
0.4979426757738342
0.7496647591996423
0.9064905781297882
0.15763598102657594
0.5111053249038157
0.8777179700937587
0.6172844516713738
0.025877138756273066


In [74]:
exe.status

(ReturnStatus(Code.DONE, None),)

## Loop with a termination condition

In [75]:
l = LoopTask()

In [76]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [77]:
l.input.control_with(
    condition=lambda task, output, scratch: output.result < .05,
    restart=lambda output, input, scratch: print(output.result)
)

In [78]:
exe = l.run()

0.3510052419168129
0.9583658097014569
0.3822386305905213
0.9446397962657214
0.5190580019824569
0.9004145100900804
0.1214505719318919
0.15782601335162105
0.2467458443276147
0.32779186092001633
0.3545881045937541
0.2742999138135499
0.1346229590051038
0.6519388412108668
0.5656985488590841


In [79]:
exe.status[0]

ReturnStatus(Code.DONE, None)

In [80]:
exe.output[0].result

0.013353255375349593